In [6]:
from datetime import date
import requests
import pandas as pd
import time
pd.set_option('display.max_columns', None)

# Profiling

In [7]:
institution_ror = catalog.load('params:openalex_extract_options.institution_ror')
env = 'dev'
print(f'institution_ror: {institution_ror}')


[09/28/25 16:09:23] INFO     Loading data from params:openalex_extract_options.institution_ror  ]8;id=848212;file:///home/pablo/dev/conicet/kedro-conicet/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=194772;file:///home/pablo/dev/conicet/kedro-conicet/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (MemoryDataset)...                                                                    

institution_ror: https://ror.org/02s7sax82


# Node

In [8]:
def openalex_extract_author(institution_ror, env):
    cursor = '*'
    base_url = 'https://api.openalex.org/authors?filter=affiliations.institution.ror:{}&cursor={}'
    iteration_limit = 5
    iteration_count = 0

    url = base_url.format(institution_ror, cursor)
    api_response = requests.get(url).json()

    print(f'Iteration count: {iteration_count}')
    print(f'GET {url}')

    # creo dataframe con las columnas del primer resultado 
    df = pd.DataFrame.from_dict(api_response['results'])

    # update cursor
    cursor = api_response['meta']['next_cursor']
    url = base_url.format(institution_ror, cursor)

    while cursor:

        if env == 'dev' and iteration_count >= iteration_limit:
            break

        # request api with updated cursor
        url = base_url.format(institution_ror, cursor)

        iteration_count += 1
        print(f'Iteration count: {iteration_count}')
        print(f'GET {url}')

        api_response = requests.get(url).json()

        df_tmp = pd.DataFrame.from_dict(api_response['results'])

        df = pd.concat([df, df_tmp])

        # update cursor
        cursor = api_response['meta']['next_cursor']
        df['load_datetime'] = date.today()

    return df

# Results

In [9]:
df = openalex_extract_author(institution_ror, env)

Iteration count: 0
GET https://api.openalex.org/authors?filter=affiliations.institution.ror:https://ror.org/02s7sax82&cursor=*
Iteration count: 1
GET https://api.openalex.org/authors?filter=affiliations.institution.ror:https://ror.org/02s7sax82&cursor=IlsyNDcsICdodHRwczovL29wZW5hbGV4Lm9yZy9BNTEwMDY3NTM4NSddIg==
Iteration count: 2
GET https://api.openalex.org/authors?filter=affiliations.institution.ror:https://ror.org/02s7sax82&cursor=IlsxODMsICdodHRwczovL29wZW5hbGV4Lm9yZy9BNTAzNDMxODgwMiddIg==
Iteration count: 3
GET https://api.openalex.org/authors?filter=affiliations.institution.ror:https://ror.org/02s7sax82&cursor=IlsxNTksICdodHRwczovL29wZW5hbGV4Lm9yZy9BNTA2OTk4NDk1MiddIg==
Iteration count: 4
GET https://api.openalex.org/authors?filter=affiliations.institution.ror:https://ror.org/02s7sax82&cursor=IlsxNDYsICdodHRwczovL29wZW5hbGV4Lm9yZy9BNTAzNzEzMDM2MCddIg==
Iteration count: 5
GET https://api.openalex.org/authors?filter=affiliations.institution.ror:https://ror.org/02s7sax82&cursor=Ilsx

In [10]:
df

,id,orcid,display_name,display_name_alternatives,works_count,cited_by_count,summary_stats,ids,affiliations,last_known_institutions,topics,topic_share,x_concepts,counts_by_year,works_api_url,updated_date,created_date,load_datetime
0,https://openalex.org/A5003447605,https://orcid.org/0000-0002-5260-1807,Gustavo E. Romero,"[Gustavo Adolfo Sierra Romero, G. E Romero, Gu...",772,9844,"{'2yr_mean_citedness': 0.9038461538461539, 'h_...",{'openalex': 'https://openalex.org/A5003447605...,[{'institution': {'id': 'https://openalex.org/...,"[{'id': 'https://openalex.org/I4210134895', 'r...","[{'id': 'https://openalex.org/T10818', 'displa...","[{'id': 'https://openalex.org/T10818', 'displa...","[{'id': 'https://openalex.org/C121332964', 'wi...","[{'year': 2025, 'works_count': 8, 'oa_works_co...",https://api.openalex.org/works?filter=author.i...,2025-09-19T09:50:53.045401,2023-07-21,2025-09-28
1,https://openalex.org/A5053874532,None,Guillermo Soberón,"[Guillermo R Soberon, Guillermo Soberon, G. So...",630,1146,"{'2yr_mean_citedness': 0.0, 'h_index': 14, 'i1...",{'openalex': 'https://openalex.org/A5053874532'},[{'institution': {'id': 'https://openalex.org/...,"[{'id': 'https://openalex.org/I4210143748', 'r...","[{'id': 'https://openalex.org/T13464', 'displa...","[{'id': 'https://openalex.org/T13403', 'displa...","[{'id': 'https://openalex.org/C138885662', 'wi...","[{'year': 2025, 'works_count': 0, 'oa_works_co...",https://api.openalex.org/works?filter=author.i...,2025-09-20T01:52:26.776348,2023-07-21,2025-09-28
2,https://openalex.org/A5112709549,https://orcid.org/0000-0002-6058-0399,Hugo Luis López,"[Hugo L. López, Hugo Luis López, Hugo L. Lopez...",484,3447,"{'2yr_mean_citedness': 1.3333333333333333, 'h_...",{'openalex': 'https://openalex.org/A5112709549...,[{'institution': {'id': 'https://openalex.org/...,"[{'id': 'https://openalex.org/I96355252', 'ror...","[{'id': 'https://openalex.org/T13589', 'displa...","[{'id': 'https://openalex.org/T13589', 'displa...","[{'id': 'https://openalex.org/C142362112', 'wi...","[{'year': 2025, 'works_count': 2, 'oa_works_co...",https://api.openalex.org/works?filter=author.i...,2025-09-25T11:33:31.188781,2024-10-15,2025-09-28
3,https://openalex.org/A5019186384,https://orcid.org/0000-0001-6247-0428,Guillermo Ricardo Simari,"[Guillermo Ricardo Simari, G. Simari, Guillerm...",467,6073,"{'2yr_mean_citedness': 1.0, 'h_index': 33, 'i1...",{'openalex': 'https://openalex.org/A5019186384...,[{'institution': {'id': 'https://openalex.org/...,"[{'id': 'https://openalex.org/I56858762', 'ror...","[{'id': 'https://openalex.org/T10456', 'displa...","[{'id': 'https://openalex.org/T10456', 'displa...","[{'id': 'https://openalex.org/C138885662', 'wi...","[{'year': 2025, 'works_count': 0, 'oa_works_co...",https://api.openalex.org/works?filter=author.i...,2025-09-21T06:55:32.687562,2023-07-21,2025-09-28
4,https://openalex.org/A5062756128,https://orcid.org/0000-0002-1030-4492,Peter Vogt,"[Peter Vogt, Peter R. Vogt, P. Vogt, V. Peter,...",421,16276,"{'2yr_mean_citedness': 5.636363636363637, 'h_i...",{'openalex': 'https://openalex.org/A5062756128...,[{'institution': {'id': 'https://openalex.org/...,"[{'id': 'https://openalex.org/I1288214837', 'r...","[{'id': 'https://openalex.org/T10001', 'displa...","[{'id': 'https://openalex.org/T14323', 'displa...","[{'id': 'https://openalex.org/C86803240', 'wik...","[{'year': 2025, 'works_count': 3, 'oa_works_co...",https://api.openalex.org/works?filter=author.i...,2025-09-21T09:10:45.305208,2023-07-21,2025-09-28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,https://openalex.org/A5016598673,https://orcid.org/0000-0003-3790-3310,M. C. Rocca,"[Mario C. Rocca, M. Rocca, Mario Rocca, M. C. ...",119,457,"{'2yr_mean_citedness': 0.0, 'h_index': 10, 'i1...",{'openalex': 'https://openalex.org/A5016598673...,[{'institution': {'id': 'https://openalex.org/...,"[{'id': 'https://openalex.org/I874386039', 'ro...","[{'id': 'https://openalex.org/T12261', 'displa...","[{'id': 'ht